## [Title in progress]

Authors: Pace, T., Anijärv, T. E., Campbell, A. J., Andrews, S.

Article link: -

Created by Toomas Erik Anijärv in 19.04.2023

This notebook is a representation of EEG processing done for the publication with one of the participants as an example.

You are free to use this or any other code from this repository for your own projects and publications. Citation or reference to the repository is not required, but would be much appreciated (see more on README.md).

In [ ]:
import mne, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from fooof import FOOOF
from fooof.plts.spectra import plot_spectrum, plot_spectrum_shading
#mne.set_log_level('error')
import warnings

# Set the current working directory to be the project main folder
os.chdir('/Users/tanijarv/Documents/GitHub/EEG-pyline')

import basic.arrange_data as arrange
import signal_processing.pre_process as pre_process
import signal_processing.spectral_analysis as spectr

**Locating the EEG files in folders** by define the experiment sub-folder (`exp_folder`), folder with raw EEG files (`raw_folder`), folder for exporting clean EEG files (`clean_folder`), and folder for exporting the results (`results_folder`). In `raw_folder` and `clean_folder`, there is `timepoints` folder which contains `exp_conditions` folder.

During pre-processing, all the raw EEG files are cleaned from the `raw_folder/exp_folder/timepoints[i]/exp_conditions[j]` and later saved to `clean_folder/exp_folder/timepoints[i]/exp_conditions[j]`. For analysis, the same clean files are read in and worked on until the results are exported to `results_folder/exp_folder`.

In [ ]:
### DEFINE ###
raw_folder = 'Data/Raw/'
clean_folder = 'Data/Clean/'
results_folder = 'Results/'

exp_folder = 'LEISURE'
timepoints = ['T1']
exp_conditions = ['EC', 'EO']

### PRE-PROCESSING

(1) The raw EEG files from `raw_folder/exp_folder` (across all timepoints and two conditions) are read in, montage is set to `biosemi32`, signals are re-referenced to `average`, and cropped to `240s` to include only the "resting" part.

(2) `0.5-30 Hz FIR filter` is designed (`zero-phase, Hamming window, order 6578`) and EOG channels are used to remove EOG-related noise with the `signal-space projections (SSP)` method.

(3) Artefact rejection is done with `Autoreject` package by removing epochs which exceed the global thereshold voltage level (`global AR`) and rest of the artefactual epochs are either removed or interpreted with `local AR`.

(4) The clean EEG signals are exported to `clean_folder/exp_folder`.

In [ ]:
### DEFINE ###
montage = 'biosemi32'
eog_channels = ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8'] # EOG channels + mastoids
stimulus_channel = 'Status'
reference = 'average'
epochs_duration = 5
filter_design = dict(l_freq=0.5,h_freq=30,filter_length='auto',method='fir',
                     l_trans_bandwidth='auto',h_trans_bandwidth='auto',
                     phase='zero',fir_window='hamming',fir_design='firwin')

In [ ]:
for timepoint in timepoints:
    print('Checking files in', timepoint)
    for exp in exp_conditions:
        # Set the directory in progress and find all BDF (raw EEG) files in there
        dir_inprogress = os.path.join(clean_folder, exp_folder, timepoint,exp)
        export_dir = os.path.join(clean_folder, exp_folder, timepoint, exp)
        file_dirs, subject_names = arrange.read_files(dir_inprogress, '.fif')

In [ ]:
timepoint = 'T1'
plot_psd = True

for exp in exp_conditions:
    print('------\nWorking with files in', timepoint, exp)
    # Set the directory in progress and find all BDF (raw EEG) files in there
    dir_inprogress = os.path.join(raw_folder, exp_folder, timepoint,exp)
    export_dir = os.path.join(clean_folder, exp_folder, timepoint, exp)
    file_dirs, subject_names = arrange.read_files(dir_inprogress, '.bdf')

    for i, file in enumerate(file_dirs):
        print('\n',i,'\n---\nCleaning EEG for', subject_names[i])
        # Read in the raw EEG data
        raw = mne.io.read_raw_bdf(file, infer_types=True, eog=eog_channels,
                                  stim_channel=stimulus_channel)

        # Set the right montage (Biosemi32) and set reference as average across all channels
        raw = raw.set_montage(mne.channels.make_standard_montage(montage)).load_data()\
                .set_eeg_reference(ref_channels=reference, verbose=False)

        # Find event markers for the start and end of resting state recordings
        events = mne.find_events(raw, stim_channel=stimulus_channel, consecutive=False, output='offset')
        
        # If there is 3 events, then crop the signal by the first and last event point
        if len(events) >= 3:
            tminmax = [events[0][0]/raw.info['sfreq'], events[-1][0]/raw.info['sfreq']]
            # If there is more than 3, warn the user (as probably requires manual processing)
            if len(events) > 3:
                warnings.warn('\nMore than 3 event points found for {}\n'.format(subject_names[i]))
        # If there is 1 or 2 event points, check whether they are start or end points or similar to each
        elif len(events) == 1 or len(events) == 2:
            warnings.warn('\nOnly 1 or 2 event point(s) found for {}\n'.format(subject_names[i]))

            if events[0][0] > 100000:
                tminmax = [0, events[0][0]/raw.info['sfreq']]
            else:
                tminmax = [events[0][0]/raw.info['sfreq'], None]
        else:
            tminmax = None
            warnings.warn('\nNO event points found for {}\n'.format(subject_names[i]))

        # Use the markers to crop to EEG signal to leave only the actual resting state
        if tminmax != None:
            cropped_raw = raw.crop(tmin=tminmax[0], tmax=tminmax[1])
            print(('Event markers are following:\n{}\nStarting point: {} s\nEnding point: {} s\n'
            'Total duration: {} s').format(events, tminmax[0], tminmax[1], tminmax[1]-tminmax[0]))
            # Warn if signal length is not what it is expected for a single condition
            if (230 <= tminmax[1]-tminmax[0] <= 250) != True:
                warnings.warn('\nRaw signal length is not between 230-250s for {}\n'.format(subject_names[i]))
        else:
            cropped_raw = raw
            print('Signal NOT cropped.')
        cropped_raw = cropped_raw.drop_channels(stimulus_channel)
        
        # Filter the signal with bandpass filter and remove EOG artefacts with SSP
        filt = pre_process.filter_raw_data(cropped_raw, filter_design, line_remove=None,
                                        eog_channels=eog_channels, plot_filt=False, savefig=False, verbose=False)
        
        if plot_psd == True: mne.viz.plot_raw_psd(filt, fmin=0, fmax=30)

        # Divide the filtered signal to epochs and run Autoreject artefact rejection on the epochs
        %matplotlib inline
        epochs = pre_process.artefact_rejection(filt, subject_names[i], epo_duration=epochs_duration, verbose=False)

        # (Optional) for displaying interactive EEG plots to visually inspect the signal quality
        #%matplotlib qt
        #epochs.plot(n_channels=32,n_epochs=1)

        # Try to create a directory and save the EEG file to that directory
        try:
            os.makedirs(export_dir)
        except FileExistsError:
            pass
        try:
            mne.Epochs.save(epochs,fname='{}/{}_clean-epo.fif'.format(export_dir,subject_names[i]),
                                                                  overwrite=True)
        except FileExistsError:
            pass

### POWER SPECTRUM DENSITY: BANDPOWERS & APERIODIC ACTIVITY

Estimation of `Welch's power spectrum density (PSD)` is done for all the participants at all timepoints, both conditions, for all 32 channls and six brain regions (i.e., `Left frontal`, `Right frontal`, `Left temporal`, `Right temporal`, `Left posterior`, `Right posterior`). The PSD estimates are divided into five frequency bands - `delta (1-3.9 Hz)`, `theta (4-7.9 Hz)`, `alpha (8-12 Hz)`, `beta (12.1-30 Hz)`.

Welch's PSD is calculated for `1-30 Hz` frequency range using `2.5-second Hamming window (50% overlap)` and 3 times the window (7.5-second) zero-padding.

Secondly, the PSDs are fitted with `specparam` (`FOOOF`) model to estimate aperiodic 1/f-like component in the spectra which can be described with parameters `exponent` and `offset`. The FOOOF (specparam) algorithm (version 1.0.0) was used to parameterize neural power spectra. Settings for the algorithm were set as: `peak width limits : 1-12 Hz`; `max number of peaks : infinite`; `minimum peak height : 0.225 uV^2`; `peak threshold : 2.0 uV^2`; and `aperiodic mode : fixed`. Power spectra were parameterized across the frequency range `1-30 Hz`. The aperiodic 1/f-like fit is described with the following function, where $S$ is aperiodic component, $b$ is `offset`, $F$ is vector of frequencies, and $e$ is `exponent`:

$S=b-log(F^e)$

The results are saved as Excel spreadsheets (channel-by channel and regionally) to `results_folder/exp_folder`.

In [ ]:
### DEFINE ###
b_names = ['Delta', 'Theta', 'Alpha', 'Beta']
b_freqs = [[1, 3.9], [4, 7.9], [8, 12], [12.1, 30]]
brain_regions = {'Left frontal' : ['AF3', 'F3', 'FC1'],
                 'Right frontal' : ['AF4', 'F4', 'FC2'],
                 'Left temporal' : ['F7', 'FC5', 'T7'],
                 'Right temporal' : ['F8', 'FC6', 'T8'],
                 'Left posterior' : ['CP5', 'P3', 'P7'],
                 'Right posterior' : ['CP6', 'P4', 'P8']}
plot_rich = True
savefig = True
psd_params = dict(method='welch', fminmax=[1, 30], window='hamming', window_duration=2.5,
                  window_overlap=0.5, zero_padding=3)
fooof_params = dict(peak_width_limits=[1,12], max_n_peaks=float("inf"), min_peak_height=0.225,
                    peak_threshold=2.0, aperiodic_mode='fixed')

# Pre-create results folders for spectral analysis data
arrange.create_results_folders(exp_folder=exp_folder, results_folder=results_folder)

sns.set_palette('muted')
sns.set_style("whitegrid")

In [ ]:
df_psd_ch_bands = pd.DataFrame()
df_fooof_regions = pd.DataFrame()
# Go through all timepoints (i.e., T1, T2, etc) AND conditions (i.e., EC, EO)
for t in range(len(timepoints)):
    for c in range(len(exp_conditions)):
        # Set the directory in progress and find all FIF (clean EEG) files in there
        dir_inprogress = os.path.join(clean_folder, exp_folder, timepoints[t], exp_conditions[c])
        file_dirs, subject_names = arrange.read_files(dir_inprogress, '_clean-epo.fif')

        for i in range(len(file_dirs)):
            # Read in the clean EEG data
            epochs = mne.read_epochs(fname='{}/{}_clean-epo.fif'.format(dir_inprogress, subject_names[i]),
                                                                        verbose=False)
            print('\n\n{} in progress'.format(subject_names[i]))

            # Calculate Welch's power spectrum density
            [psds,freqs] = spectr.calculate_psd(epochs, subject_names[i], **psd_params,
                                                verbose=True, plot=False)
            
            # BANDPOWERS CALCULATION
            for j in range(len(b_names)):
                # Devide the PSD to frequency band bins and calculate absolute bandpowers incl. signal quality check
                psd_ch_band_temp = spectr.bandpower_per_channel(psds, freqs, b_freqs[j], b_names[j],
                                                        subject_names[i], epochs)
                # Convert the array to dataframe and add the corresponding band, timepoint and condition
                df_psd_ch_band_temp = arrange.array_to_df(subject_names[i], epochs, psd_ch_band_temp)
                df_psd_ch_band_temp.insert(0, 'Condition', exp_conditions[c]) # FIX!
                df_psd_ch_band_temp.insert(1, 'Timepoint', timepoints[t]) # FIX!
                df_psd_ch_band_temp.insert(2, 'Measure', b_names[j])

                # Concatenate it to the dataframe including all the previous subjects
                df_psd_ch_bands = pd.concat([df_psd_ch_bands, df_psd_ch_band_temp])
            
            # APERIODIC ACTIVITY ESTIMATION
            # Average all epochs together for each channel and also for each region
            psds = psds.mean(axis=(0))
            df_psds_ch = arrange.array_to_df(subject_names[i], epochs, psds).\
                                    reset_index().drop(columns='Subject')
            df_psds_regions = arrange.df_channels_to_regions(df_psds_ch, brain_regions).\
                                        reset_index().drop(columns='Subject')

            # Go through all regions of interest
            df_fooof_regs_temp = pd.DataFrame()
            for region in df_psds_regions.columns:
                psds_temp = df_psds_regions[region].to_numpy()

                # Fit the spectrum with FOOOF (specparam)
                fm = FOOOF(**fooof_params, verbose=True)
                fm.fit(freqs, psds_temp, psd_params['fminmax'])

                # Set plot styles
                data_kwargs = {'color' : 'black', 'linewidth' : 1.4, 'label' : 'Original'}
                model_kwargs = {'color' : 'red', 'linewidth' : 1.4, 'alpha' : 0.75, 'label' : 'Full model'}
                aperiodic_kwargs = {'color' : 'blue', 'linewidth' : 1.4, 'alpha' : 0.75,
                                    'linestyle' : 'dashed', 'label' : 'Aperiodic model'}
                
                # Plot power spectrum model + aperiodic fit
                fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6, 4), dpi=100)
                plot_spectrum(fm.freqs, fm.power_spectrum,
                            ax=axs, plot_style=None, **data_kwargs)
                plot_spectrum(fm.freqs, fm.fooofed_spectrum_,
                            ax=axs, plot_style=None, **model_kwargs)
                plot_spectrum(fm.freqs, fm._ap_fit,
                            ax=axs, plot_style=None, **aperiodic_kwargs)
                axs.set_xlim(psd_params['fminmax'])
                axs.grid(linewidth=0.2)
                axs.set_xlabel('Frequency (Hz)')
                axs.set_ylabel('Log-normalised power (log10[uV\u00b2/Hz])')
                axs.set_title('Spectrum model fit')
                axs.legend()

                # If true, plot all the exported variables on the plots
                if plot_rich == True:
                    axs.annotate('Error: ' + str(np.round(fm.get_params('error'), 4)) +
                                '\nR\u00b2: ' + str(np.round(fm.get_params('r_squared'), 4)),
                                (0.15, 0.16), xycoords='figure fraction', color='red', fontsize=8.5)
                    axs.annotate('Exponent: ' + str(np.round(fm.get_params('aperiodic_params','exponent'), 4)) +
                                '\nOffset: ' + str(np.round(fm.get_params('aperiodic_params','offset'), 4)),
                                (0.29, 0.16), xycoords='figure fraction', color='blue', fontsize=8.5)
                
                plt.suptitle('{} region ({})'.format(region, subject_names[i]))
                plt.tight_layout()
                if savefig == True:
                    plt.savefig(fname='{}/{}/FOOOF/{}_{}_fooof.png'.format(results_folder, exp_folder,
                                                                           subject_names[i], region), dpi=300)
                
                # Plot the figure only if the following criterias apply (meaning the fit should be visually checked)
                if (fm.get_params('aperiodic_params','exponent') <= 0) or (fm.get_params('r_squared') < 0.6) or \
                   (fm.get_params('error') > 0.15):
                    plt.show()
                else:
                    plt.close()

                # Extract exponent and offset for the current subject in current region
                df_fooof_reg_temp = pd.DataFrame(data={region: [fm.get_params('aperiodic_params','exponent'),
                                                                fm.get_params('aperiodic_params','offset'),
                                                                fm.get_params('r_squared'),
                                                                fm.get_params('error')]},
                                                index=[subject_names[i], subject_names[i],
                                                       subject_names[i], subject_names[i]])

                # Add the region data to all regions dataframe for the current subject
                df_fooof_regs_temp = pd.concat([df_fooof_regs_temp, df_fooof_reg_temp], axis=1)
            
            # Add additional information about the condition, timepoint, and which is which measure
            df_fooof_regs_temp.insert(0, 'Condition', exp_conditions[c])
            df_fooof_regs_temp.insert(1, 'Timepoint', timepoints[t])
            df_fooof_regs_temp.insert(2, 'Measure', ['Exponent', 'Offset', 'R_2', 'Error'])

            # Merge all subjects' data together to single dataframe
            df_fooof_regions = pd.concat([df_fooof_regions, df_fooof_regs_temp])
            df_fooof_regions.index.name = 'Subject'
            
# Average the channels together for the specified regions (bandpowers)
df_psd_reg_bands = arrange.df_channels_to_regions(df_psd_ch_bands.reset_index(), brain_regions)
df_psd_reg_bands = df_psd_reg_bands.set_index(df_psd_ch_bands.index)
df_psd_reg_bands.insert(0, 'Condition', df_psd_ch_bands['Condition'])
df_psd_reg_bands.insert(1, 'Timepoint', df_psd_ch_bands['Timepoint'])
df_psd_reg_bands.insert(2, 'Measure', df_psd_ch_bands['Measure'])

# Display and export the final bandpower results
display(df_psd_ch_bands)
display(df_psd_reg_bands)
df_psd_ch_bands.to_excel('{}/{}/{}_channels_bandpowers.xlsx'.format(results_folder, exp_folder, exp_folder))
df_psd_reg_bands.to_excel('{}/{}/{}_regions_bandpowers.xlsx'.format(results_folder, exp_folder, exp_folder))

# Display and export the final aperiodic activity results
display(df_fooof_regions)
df_fooof_regions.to_excel('{}/{}/{}_regions_aperiodiccomponents.xlsx'.format(results_folder, exp_folder, exp_folder))

Removed participants for aperiodic components dataframe are following:

**EC:**

`HBA_0007_EC_T1` // bad model fit (exp<=0)

`HBA_0009_EC_T1` // bad model fit (exp<=0)

`HBA_0016_EC_T1` // bad model fit (exp<=0)

**EO:**

`HBA_0001_EO_T1` // bad model fit (exp<=0)

`HBA_0007_EO_T1` // bad model fit (exp<=0)

`HBA_0009_EO_T1` // bad model fit (exp<=0)

`HBA_0013_EO_T1` // bad model fit (R_2<0.2)

`HBA_0014_EO_T1` // bad model fit (R_2<0.2)

`HBA_0025_EO_T1` // bad model fit (exp<=0)

`HBA_0042_EO_T1` // bad model fit (exp<=0)

`HBA_0058_EO_T1` // bad model fit (exp<=0)

`HBA_0119_EO_T1` // bad model fit (exp<=0)

`HBA_0121_EO_T1` // bad model fit (exp<=0)

In [97]:
# Participants chosen visually to be removed due to either bad model fit or alpha peak detected wrongly
bad_participants = ['HBA_0007_EC_T1', 'HBA_0009_EC_T1', 'HBA_0016_EC_T1',
                    'HBA_0001_EO_T1', 'HBA_0007_EO_T1', 'HBA_0009_EO_T1',
                    'HBA_0013_EO_T1', 'HBA_0014_EO_T1', 'HBA_0025_EO_T1',
                    'HBA_0042_EO_T1', 'HBA_0058_EO_T1', 'HBA_0119_EO_T1',
                    'HBA_0121_EO_T1']

# Remove the bad participants from the results dataframe AND PSDs dataframe AND demo/att dataframe
df_fooof_regions_wo_bads = df_fooof_regions.drop(index=(bad_participants))

# Export the master dataframes that do not have the "bad" participants
df_fooof_regions_wo_bads.to_excel('{}/{}/{}_regions_aperiodiccomponents_without_bad_fits.xlsx'.format(results_folder, exp_folder, exp_folder))

In [98]:
display(df_psd_ch_bands)
display(df_psd_reg_bands)
display(df_fooof_regions_wo_bads)

,Subject,Condition,Timepoint,Measure,Fp1,AF3,F7,F3,FC1,FC5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,HBA_0001_EC_T1,EC,T1,Delta,4.629211,3.834887,4.630041,1.355184,2.149151,2.056162,...,0.776797,2.565394,1.252406,0.792816,0.776881,3.757014,1.440673,5.260071,0.692926,0.567881
1,HBA_0001_EC_T1,EC,T1,Theta,0.784446,1.056541,1.300763,0.563286,0.724476,0.749780,...,0.732600,0.848106,0.611284,0.533311,0.583241,0.868721,0.537351,0.722373,0.510169,0.345089
2,HBA_0001_EC_T1,EC,T1,Alpha,2.450612,1.968534,3.693824,1.206546,0.730973,2.131703,...,2.034198,2.483673,1.661508,0.585043,1.007888,2.326043,1.406513,1.994050,0.874559,0.717087
3,HBA_0001_EC_T1,EC,T1,Beta,0.728894,0.788438,0.972776,0.662868,0.497620,0.998694,...,1.433036,1.218004,1.490550,0.417982,0.742062,1.942821,0.633489,0.659459,0.562728,0.535782
4,HBA_0003_EC_T1,EC,T1,Delta,7.695061,3.360140,5.162439,1.485254,1.452389,1.476928,...,4.036212,2.925764,2.544133,3.092440,1.428658,3.013939,3.730808,7.901654,1.491102,1.433264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,HBA_0129_EO_T1,EO,T1,Beta,0.531836,0.193142,0.562001,0.576211,0.495958,0.798113,...,0.867025,0.939295,0.969336,0.497808,0.855124,0.443305,0.257545,0.510557,0.437856,0.369355
784,HBA_0130_EO_T1,EO,T1,Delta,1.739652,1.089388,4.359820,3.528685,5.326037,3.682670,...,1.957366,7.303896,2.743244,2.310762,2.885623,2.081006,0.984736,1.895492,1.941399,2.337281
785,HBA_0130_EO_T1,EO,T1,Theta,0.437784,0.304670,1.134639,0.917123,1.200360,0.930783,...,0.487578,1.337073,0.667863,0.572474,0.748403,0.674675,0.257807,0.440868,0.605183,0.469188
786,HBA_0130_EO_T1,EO,T1,Alpha,0.404397,0.213786,1.169921,0.622751,0.655229,0.705899,...,0.396320,0.932589,0.525842,0.416362,0.490149,0.631663,0.207933,0.378746,0.348090,0.431791


,Subject,Condition,Timepoint,Measure,Left frontal,Right frontal,Left temporal,Right temporal,Left posterior,Right posterior
0,HBA_0001_EC_T1,EC,T1,Delta,2.446407,1.003457,3.150284,2.524938,1.666004,1.643361
1,HBA_0001_EC_T1,EC,T1,Theta,0.781435,0.551301,0.980835,0.776037,0.669002,0.711884
2,HBA_0001_EC_T1,EC,T1,Alpha,1.302018,0.999815,3.124136,2.157075,2.306400,2.708413
3,HBA_0001_EC_T1,EC,T1,Beta,0.649642,0.597845,0.923992,1.550458,0.644241,0.831289
4,HBA_0003_EC_T1,EC,T1,Delta,2.099261,2.750635,3.076996,2.827945,2.575220,1.816120
...,...,...,...,...,...,...,...,...,...,...
783,HBA_0129_EO_T1,EO,T1,Beta,0.421770,0.536826,0.739113,0.783979,0.592833,0.699595
784,HBA_0130_EO_T1,EO,T1,Delta,3.314703,2.060374,3.544619,4.042715,4.602580,2.365640
785,HBA_0130_EO_T1,EO,T1,Theta,0.807385,0.526228,0.897594,0.893204,0.925955,0.556127
786,HBA_0130_EO_T1,EO,T1,Alpha,0.497256,0.371481,0.914226,0.696698,0.646491,0.453478


,Condition,Timepoint,Measure,Left frontal,Right frontal,Left temporal,Right temporal,Left posterior,Right posterior
Subject,,,,,,,,,
HBA_0001_EC_T1,EC,T1,Exponent,0.796945,0.571945,0.613370,0.211913,0.658756,0.669491
HBA_0001_EC_T1,EC,T1,Offset,0.562065,0.118805,0.574402,0.263329,0.376160,0.375538
HBA_0001_EC_T1,EC,T1,R_2,0.945886,0.947832,0.913391,0.657026,0.962926,0.967469
HBA_0001_EC_T1,EC,T1,Error,0.054064,0.055883,0.057785,0.087002,0.048606,0.050842
HBA_0003_EC_T1,EC,T1,Exponent,0.927303,0.908721,0.635870,0.480628,0.763067,0.855977
...,...,...,...,...,...,...,...,...,...
HBA_0129_EO_T1,EO,T1,Error,0.066256,0.057751,0.041244,0.031726,0.043597,0.039162
HBA_0130_EO_T1,EO,T1,Exponent,0.818968,0.843825,0.680680,0.806425,1.082984,0.945321
HBA_0130_EO_T1,EO,T1,Offset,0.611191,0.437779,0.554483,0.685917,0.904119,0.554056


### ALPHA REACTIVITY

`Alpha reactivity` is calculated with the following formula:
$$Alpha reactivity = (EC alpha power - EO alpha power) / EC alpha power$$

The results are saved as Excel spreadsheets (channel-by channel and regionally) to `results_folder/exp_folder`.

In [99]:
df_psd_ch_bands = pd.read_excel('{}/{}/{}_channels_bandpowers.xlsx'.format(results_folder, exp_folder, exp_folder))
df_psd_reg_bands = pd.read_excel('{}/{}/{}_regions_bandpowers.xlsx'.format(results_folder, exp_folder, exp_folder))

In [100]:
def alpha_reactivity(df_psd_bands):
    # Choose only alpha band from the bandpowers dataframe
    df_psd_alpha = df_psd_bands[df_psd_bands['Measure']=='Alpha']

    # Divide the alpha dataframe to EC and EO conditions
    df_psd_alpha_EC = df_psd_alpha[df_psd_alpha['Condition']=='EC'].reset_index(drop=True)
    df_psd_alpha_EC['Subject'] = df_psd_alpha_EC['Subject'].str.removesuffix('_EC_T1')
    df_psd_alpha_EO = df_psd_alpha[df_psd_alpha['Condition']=='EO'].reset_index(drop=True)
    df_psd_alpha_EO['Subject'] = df_psd_alpha_EO['Subject'].str.removesuffix('_EO_T1')
    
    # Merge the two conditions together to one dataframe by only including subjects who have both conditions
    df_psd_alpha_EC_EO = df_psd_alpha_EC.merge(df_psd_alpha_EO, on='Subject', how='inner', suffixes=('_EC', '_EO'))

    # Create new dataframe and calculate alpha reactivity for each subject
    df_psd_alpha_reactivity = pd.DataFrame(columns=df_psd_bands.columns).drop(columns=['Condition', 'Measure'])
    df_psd_alpha_reactivity['Subject'] = df_psd_alpha_EC_EO['Subject']
    df_psd_alpha_reactivity['Timepoint'] = df_psd_alpha_EC_EO['Timepoint_EC']
    for ch in df_psd_alpha_reactivity.columns[2:]:
        df_psd_alpha_reactivity['{}'.format(ch)] = (df_psd_alpha_EC_EO['{}_EC'.format(ch)] - df_psd_alpha_EC_EO['{}_EO'.format(ch)]) / df_psd_alpha_EC_EO['{}_EC'.format(ch)]
    df_psd_alpha_reactivity.insert(2, 'Measure', 'Alpha reactivity')
    
    return df_psd_alpha_reactivity

In [101]:
df_psd_ch_alpha_reactivity = alpha_reactivity(df_psd_ch_bands).set_index(['Subject'])
df_psd_reg_alpha_reactivity = alpha_reactivity(df_psd_reg_bands).set_index(['Subject'])

display(df_psd_ch_alpha_reactivity)
display(df_psd_reg_alpha_reactivity)

df_psd_ch_alpha_reactivity.to_excel('{}/{}/{}_channels_alpha_reactivity.xlsx'.format(results_folder, exp_folder, exp_folder))
df_psd_reg_alpha_reactivity.to_excel('{}/{}/{}_regions_alpha_reactivity.xlsx'.format(results_folder, exp_folder, exp_folder))

,Timepoint,Measure,Fp1,AF3,F7,F3,FC1,FC5,T7,C3,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
Subject,,,,,,,,,,,,,,,,,,,,,
HBA_0001,T1,Alpha reactivity,0.716294,0.703207,0.608576,0.029135,-0.541415,0.323622,0.614755,0.106982,...,0.178159,0.388086,0.292180,-0.249472,0.378722,0.531938,0.856587,0.846055,0.387347,-0.175841
HBA_0003,T1,Alpha reactivity,0.789568,0.887007,0.566013,0.817636,0.697663,0.558335,0.295470,0.631559,...,0.810200,0.280272,0.640728,0.666177,0.793157,0.546957,0.846853,0.673264,0.843975,0.619396
HBA_0004,T1,Alpha reactivity,-0.155038,0.781695,-1.052941,0.518361,0.494425,-0.876991,-0.617731,0.222883,...,0.347120,-0.322506,0.235874,0.513320,0.624207,0.174318,0.741043,0.352776,0.686509,0.398419
HBA_0005,T1,Alpha reactivity,0.528772,0.726453,0.498821,-0.033347,-1.169923,0.397312,0.546322,0.060017,...,-1.422474,0.444474,0.241341,-1.264439,-0.428798,0.533609,0.684560,0.618652,-0.708965,0.159739
HBA_0006,T1,Alpha reactivity,0.400135,0.564371,0.393298,-0.338259,-2.693184,-0.024015,0.068922,0.070037,...,-0.348502,0.102092,0.147681,-1.311971,-0.104090,0.414017,0.404954,0.378186,-0.960632,-1.267431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HBA_0122,T1,Alpha reactivity,0.295322,-0.770125,-0.223599,-0.315739,0.142664,0.019999,0.104227,0.311098,...,0.172015,0.170842,0.173493,0.199713,0.140826,0.198550,0.211908,0.275021,0.081622,0.191933
HBA_0126,T1,Alpha reactivity,-0.017579,-0.293427,0.121843,-0.103243,0.757781,0.052121,0.282678,0.150335,...,0.272914,0.408800,-0.153734,0.524477,-0.331697,0.281493,-0.325272,0.076201,0.353692,0.744767
HBA_0128,T1,Alpha reactivity,0.099672,0.466137,-0.383882,0.225272,0.154723,-0.101421,0.051219,0.352327,...,0.416302,0.108393,0.229420,0.328304,0.115806,0.032630,0.382216,0.460133,0.295006,0.320746


,Timepoint,Measure,Left frontal,Right frontal,Left temporal,Right temporal,Left posterior,Right posterior
Subject,,,,,,,,
HBA_0001,T1,Alpha reactivity,0.262075,0.480275,0.546103,0.415169,0.693355,0.636418
HBA_0003,T1,Alpha reactivity,0.801462,0.763590,0.463580,0.487968,0.665767,0.833154
HBA_0004,T1,Alpha reactivity,0.580789,0.615372,-0.750826,-0.027850,0.403286,0.465100
HBA_0005,T1,Alpha reactivity,0.170187,0.014898,0.502251,0.438689,0.708471,0.703104
HBA_0006,T1,Alpha reactivity,-0.314324,-0.117885,0.195830,0.250193,0.273268,0.129630
...,...,...,...,...,...,...,...,...
HBA_0122,T1,Alpha reactivity,-0.111161,0.179550,-0.009117,0.179338,0.238049,0.359358
HBA_0126,T1,Alpha reactivity,0.074393,0.016418,0.178981,0.300763,0.269379,0.188872
HBA_0128,T1,Alpha reactivity,0.225902,0.271889,-0.075366,0.123724,0.370339,0.312974
